In [6]:
import os, pandas, numpy
from sklearn.model_selection import KFold
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [20]:

df1=pandas.read_csv("PS_20174392719_1491204439457_log.csv", na_values=['NA', '?'])
pandas.set_option('display.max_columns', None)

df1

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [21]:
df1=df1.reindex(numpy.random.permutation(df1.index))
mask=numpy.random.rand(len(df1))<0.8
trainDF=pandas.DataFrame(df1[mask])
validationDF=pandas.DataFrame(df1[~mask])

print(f"Training DF: {len(trainDF)}")
print(f"Validation DF: {len(validationDF)}")

Training DF: 5089921
Validation DF: 1272699


In [22]:
df2=pandas.read_csv("PS_20174392719_1491204439457_log.csv", na_values=['NA', '?'])
df2=df2.reindex(numpy.random.permutation(df2.index))
kf =KFold(5)

fold = 1
for train_index, validate_index in kf.split(df2):
    trainDF=pandas.DataFrame(df2.iloc[train_index, :])
    validateDF=pandas.DataFrame(df2.iloc[validate_index])
    print(f"Fold#{fold}, Training Size: {len(trainDF)},  Validation Size: {len(validationDF)}")
    fold+=1

Fold#1, Training Size: 5090096,  Validation Size: 1272699
Fold#2, Training Size: 5090096,  Validation Size: 1272699
Fold#3, Training Size: 5090096,  Validation Size: 1272699
Fold#4, Training Size: 5090096,  Validation Size: 1272699
Fold#5, Training Size: 5090096,  Validation Size: 1272699


In [3]:
#loading the dataset as dataframe
df3=pandas.read_csv("PS_20174392719_1491204439457_log.csv", na_values=['NA', '?'])

#shuffling the data
df3=df3.reindex(numpy.random.permutation(df3.index))

enc=LabelEncoder()
column_name= "type"
space=1
enc.fit(df3[column_name])
enc_type = enc.transform(df3[column_name])
df3.insert(space,"encoded_" + column_name,enc_type)
df3.pop("type")

column_name= "nameOrig"
space=3
enc.fit(df3[column_name])
enc_type = enc.transform(df3[column_name])
df3.insert(space,"encoded_" + column_name,enc_type)
df3.pop("nameOrig")

column_name= "nameDest"
space=6
enc.fit(df3[column_name])
enc_type = enc.transform(df3[column_name])
df3.insert(space,"encoded_" + column_name,enc_type)
df3.pop("nameDest")


result = []
for x in df3.columns:
    if x != 'isFraud':
        result.append(x)

# x and y data
X=df3[result].values
y=df3['isFraud'].values

#Perceptron creation
ppn=Perceptron(max_iter=70, tol=0.0001, eta0=1)

#k-fold splitting to 5(for now, might be good with 10 as well)
kf =KFold(10)

fold = 1
for train_index, validate_index in kf.split(X, y):
    ppn.fit(X[train_index], y[train_index])
    y_test=y[validate_index]
    y_pred=ppn.predict(X[validate_index])
    
    
    print(f"Fold#{fold}, Training Size: {len(X[train_index])},  Validation Size: {len(X[validate_index])}")
    print("Accuracy: %.2f" % accuracy_score(y_test, y_pred))
    
    #precision check
    precision = metrics.average_precision_score(y_test, y_pred)
    print("precision score: {}".format(precision))
    
    #validateDF=pandas.DataFrame(df3.iloc[validate_index])
    #print(f"Fold#{fold}, Training Size: {len(trainDF)},  Validation Size: {len(validationDF)}")
    fold+=1

Fold#1, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.144868346800763
Fold#2, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.3549610809305591
Fold#3, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.37416001001714094
Fold#4, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.19994979316140254
Fold#5, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.16164012647347104
Fold#6, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.23181377585081087
Fold#7, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.30368465719620297
Fold#8, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.2575257694094637
Fold#9, Training Size: 5726358,  Validation Size: 636262
Accuracy: 1.00
precision score: 0.24681635498426863
Fold#10, Training Size:

In [30]:
df3

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [35]:
#this shows that there are no missing values which need to be replaced
print(df3.isnull().any())

#this shows that the dataset is unbalanced and needs some manipulation
print(df3["isFraud"].value_counts())
#this shows that the dataset unbalanced in percentage 99.87 to 0.0012
print(df3["isFraud"].value_counts(normalize=True))


step              False
type              False
amount            False
nameOrig          False
oldbalanceOrg     False
newbalanceOrig    False
nameDest          False
oldbalanceDest    False
newbalanceDest    False
isFraud           False
isFlaggedFraud    False
dtype: bool
0    6354407
1       8213
Name: isFraud, dtype: int64
0    0.998709
1    0.001291
Name: isFraud, dtype: float64


In [33]:
df_positive=df3.loc[df3["isFraud"]==1]
df_negative=df3.loc[df3["isFraud"]==0]
dfSmall= pandas.concat([df_positive, df_negative])
dfSmall

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1,0
251,1,TRANSFER,2806.00,C1420196421,2806.0,0.00,C972765878,0.00,0.00,1,0
252,1,CASH_OUT,2806.00,C2101527076,2806.0,0.00,C1007251739,26202.00,0.00,1,0
680,1,TRANSFER,20128.00,C137533655,20128.0,0.00,C1848415041,0.00,0.00,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6362319,718,PAYMENT,8634.29,C642813806,518802.0,510167.71,M747723689,0.00,0.00,0,0
6362320,718,CASH_OUT,159188.22,C691808084,3859.0,0.00,C1818183087,0.00,159188.22,0,0
6362321,718,CASH_OUT,186273.84,C102120699,168046.0,0.00,C1515639522,24893.67,211167.51,0,0
6362322,718,TRANSFER,82096.45,C614459560,13492.0,0.00,C855350324,0.00,82096.45,0,0
